# scrape topic index


In [2]:
%pip install chromadb
%pip install bs4
%pip install tensorflow
%pip install tensorflow_hub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Linus\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Linus\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Linus\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Linus\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
import requests
import time
import chromadb
from bs4 import BeautifulSoup
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np

In [5]:
#load wikipedia article titles

wiki_titles = []
with open("./data/enwiki-latest-all-titles.txt", "r", encoding="utf-8") as f:
    for line in f:
        wiki_titles.append(line.strip())

print(len(wiki_titles))

17436825


In [15]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [22]:
def custom_ef(text):
    embeddings = embed(text)
    
    #flatten tensor to list
    embeddings = np.array(embeddings).flatten().tolist()
    
    return embeddings

In [49]:
def time_to_str(t: int):
    """Converts time in seconds to the appropriate time unit"""
    
    if t > 3600 * 24:
        out_str = f"{round(t / (3600 * 24), 2)} days"
    elif t > 3600:
        out_str = f"{round(t / 3600, 2)} hours"
    elif t > 60:
        out_str = f"{round(t / 60, 2)} minutes"
    elif 60 > t > 1:
        out_str = f"{round(t, 2)} seconds"
    elif t < 1:
        out_str = f"{t * 1000} milliseconds"
    else:
        out_str = f"{round(t, 2)} seconds"
    
    return out_str

In [64]:
#compute all embeddings
wiki_embeddings = []
len_titles = len(wiki_titles)
t_start = time.time()
temp = []
for idx, title in enumerate(wiki_titles):
    temp.append(custom_ef([title]))
    
    if idx % 100 == 0:
        
        wiki_titles += temp
        temp = []
        
        time_taken = time.time() - t_start
        time_per_title = time_taken / (idx + 1)
        time_remaining = time_per_title * (len_titles - idx)
        
        percentage_done = round(idx / len_titles * 100, 2)
        
        print(" " * 100, end="\r")
        print(f"Processed {idx}/{len_titles} ({percentage_done}%) titles in {time_to_str(time_taken)}. {time_to_str(time_remaining)} remaining.", end="\r")

KeyboardInterrupt: 

In [19]:
Client = chromadb.PersistentClient("./data/chromadb")

In [20]:
#delete collection "wiki_titles"

Client.delete_collection("wiki_titles")

ValueError: Collection wiki_titles does not exist.

In [26]:
#add wikipedia article titles to chromadb

collection = Client.get_or_create_collection("wiki_titles")



Batch 1 / 34873 (311.8 elements/s, 15.5 hours remaining)
Batch 2 / 34873 (342.8 elements/s, 14.1 hours remaining)
Batch 3 / 34873 (338.6 elements/s, 14.3 hours remaining)
Batch 4 / 34873 (337.1 elements/s, 14.4 hours remaining)
Batch 5 / 34873 (358.7 elements/s, 13.5 hours remaining)
Batch 6 / 34873 (351.7 elements/s, 13.8 hours remaining)
Batch 7 / 34873 (355.2 elements/s, 13.6 hours remaining)
Batch 8 / 34873 (336.7 elements/s, 14.4 hours remaining)
Batch 9 / 34873 (320.7 elements/s, 15.1 hours remaining)
Batch 10 / 34873 (334.7 elements/s, 14.5 hours remaining)
Batch 11 / 34873 (358.5 elements/s, 13.5 hours remaining)
Batch 12 / 34873 (348.5 elements/s, 13.9 hours remaining)
Batch 13 / 34873 (356.8 elements/s, 13.6 hours remaining)
Batch 14 / 34873 (336.4 elements/s, 14.4 hours remaining)
Batch 15 / 34873 (349.6 elements/s, 13.8 hours remaining)
Batch 16 / 34873 (323.0 elements/s, 15.0 hours remaining)
Batch 17 / 34873 (354.2 elements/s, 13.7 hours remaining)
Batch 18 / 34873 (337.9

KeyboardInterrupt: 

In [ ]:
def query(query: str, num_results: int = 10):
    collection = Client.get_collection("wiki_titles")
    
    t_start = time.time()
    
    r = collection.query(
        query_texts=[query],
        n_results=num_results
    )
    
    return r